# 피마 인디언 당뇨병 예측

In [1]:
import pandas as pd
import numpy as np

- 데이터 전처리

In [4]:
from google.colab import files
up = files.upload()

Saving pima-indians-diabetes.csv to pima-indians-diabetes.csv


In [6]:
df = pd.read_csv('pima-indians-diabetes.csv', skiprows=9, header=None)
df.columns = ['P', 'G', 'BP', 'S', 'I', 'BMI', 'D', 'Age', 'Class']
df.head()

,P,G,BP,S,I,BMI,D,Age,Class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
# NaN(결측지) 여부 확인
df.isna().sum().sum() # 모든 컬럼을 통틀어 결측지가 없는지 한 번에 파악하는 방법

0

In [8]:
# X, y를 ndarray로 추출
X = df.iloc[: , :-1].values # 2차원
y = df.iloc[: , -1].values # 1차원

In [9]:
X.shape, y.shape

((768, 8), (768,))

##### Train, Test dataset으로 분리

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify = y, test_size = 0.2, random_state = 2022
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

In [11]:
np.unique(y_train, return_counts = True) # 훈련용 데이터가 잘 나뉘었는지 확인

(array([0, 1]), array([400, 214]))

In [13]:
np.unique(y_test, return_counts = True) # 훈련용 데이터가 잘 나뉘었는지 확인

(array([0, 1]), array([100,  54]))

- Decision Tree Classifier

In [14]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=2022)
dtc.get_params() # 구성 인수 확인용

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 2022,
 'splitter': 'best'}

In [15]:
params = {'max_depth': [2,3,4,5,6], 'min_samples_split':[2,3,4]}

- GridSearchCV

In [16]:
from sklearn.model_selection import GridSearchCV
grid_dt = GridSearchCV(dtc, params, scoring='accuracy', cv=5)
grid_dt.fit(X_train, y_train) # GridSearchCV로 학습

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=2022),
             param_grid={'max_depth': [2, 3, 4, 5, 6],
                         'min_samples_split': [2, 3, 4]},
             scoring='accuracy')

In [17]:
grid_dt.best_params_ # 학습하는 데 필요한 최적 인수 도출

{'max_depth': 5, 'min_samples_split': 3}

In [18]:
best_dt = grid_dt.best_estimator_
best_dt.score(X_test, y_test)

0.6558441558441559

- 실제 적용

In [19]:
X_test[0], y_test[0]

(array([  0.   , 135.   ,  94.   ,  46.   , 145.   ,  40.6  ,   0.284,
         26.   ]), 0)

In [ ]:
# 아래처럼 입력하면 테스트 데이터 차원이 맞지 않아 에러남
# best_dt.predict(X_test[0])

In [20]:
best_dt.predict(X_test)

array([0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [22]:
X_train.shape, X_test[0].shape

((614, 8), (8,))

In [23]:
best_dt.predict(X_test[0].reshape(1,-1))

array([0])

In [24]:
pred = best_dt.predict(X_test[0].reshape(1,-1))
print('양성' if pred[0] == 1 else '음성')

음성
